In [8]:
%pip install scipy==1.7.3
%pip install imageai --upgrade
%pip install split-folders
%pip install tensorflow==2.2.0
%pip install pandas
%pip install keras==2.4.3

import PIL
import os
# https://github.com/jfilter/split-folders
import splitfolders
# https://github.com/OlafenwaMoses/ImageAI
from imageai import DetectionModelTrainer
from imageai import CustomObjectDetection


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.2.1 -> 22.2.2
[notice] To update, run: C:\Users\Ruben\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


  Using cached imageai-2.1.6-py3-none-any.whl (160 kB)
  Using cached numpy-1.19.3.zip (7.3 MB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'error'
Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  × Preparing metadata (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [202 lines of output]
      setup.py:67: RuntimeWarning: NumPy 1.19.3 may not yet support Python 3.10.
        warnings.warn(
      Running from numpy source directory.
      setup.py:480: UserWarning: Unrecognized setuptools command, proceeding with generating Cython sources and expanding templates
        run_build = parse_setuppy_commands()
      Cythonizing sources
      Processing numpy/random\_bounded_integers.pxd.in
      Processing numpy/random\bit_generator.pyx
      Processing numpy/random\mtrand.pyx
      Processing numpy/random\_bounded_integers.pyx.in
      Processing numpy/random\_common.pyx
      Processing numpy/random\_generator.pyx
      Processing numpy/random\_mt19937.pyx
      Processing numpy/random\_pcg64.pyx
      Processing numpy/random\_philox.pyx
      Processing numpy/random\_sfc64.pyx
      blas_opt_info:
      blas_mkl_info:
   

Note: you may need to restart the kernel to use updated packages.


...

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.2.1 -> 22.2.2
[notice] To update, run: C:\Users\Ruben\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.2.1 -> 22.2.2
[notice] To update, run: C:\Users\Ruben\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


ModuleNotFoundError: No module named 'imageai'

In [ ]:
dataset_folder="MyDataset"
model_folder="MyModels"
directory_name = "MyDetection"

input_folder = f"/content/{dataset_folder}"
output = f"/content/{directory_name}"
splitfolders.ratio(input_folder, output=output, seed=42, ratio=(.70, .15, .15))

# Download pretrained model
!wget https://github.com/OlafenwaMoses/ImageAI/releases/download/essential-v4/pretrained-yolov3.h5 /content/$directory_name

In [ ]:
# Setup trainer
trainer = DetectionModelTrainer()  
trainer.setModelTypeAsYOLOv3()
trainer.setDataDirectory(data_directory=output)
trainer.setTrainConfig(object_names_array=['plastic', 'other'],
                       batch_size=4, num_experiments=20,
                       train_from_pretrained_model="pretrained-yolov3.h5"
                       )
trainer.trainModel()

In [ ]:
# Image to test
image_path = f'/content/{directory_name}/test/images'
image_name = os.listdir(image_path)[0]
# Minimum detection confidence threshold percentage. If you don't get any detections, try lowering the threshold. 
min_threshold = 30

# Get our model 
!cp -r /content/$model_folder/* /content/$directory_name/models

# Get the latest model saved (with the lowest loss)
all_models = os.listdir(f'/content/{directory_name}/models')
all_models.sort()
best_model = all_models[-1]

detector = CustomObjectDetection()
detector.setModelTypeAsYOLOv3()
detector.setModelPath(f"/content/{directory_name}/models/{best_model}")
detector.setJsonPath(f"/content/{directory_name}/json/detection_config.json")
detector.loadModel()
detections = detector.detectObjectsFromImage(input_image=f"{image_path}/{image_name}",
                                             output_image_path="/content/detections.jpg",
                                             minimum_percentage_probability=min_threshold)

# Show image with detections
image = PIL.Image.open("/content/detections.jpg")
display(image)